In [1]:
import numpy as np
import pandas as pd
import pickle
import xgboost
import os
from scipy.stats import ttest_1samp, ttest_ind, ttest_rel
from config import path_prefix_results
import warnings

warnings.filterwarnings('ignore')


## Create latex template for thigh, breast results

In [2]:
#latex output setup
latex_template = r'''\documentclass[a4paper]{{article}}
\usepackage{{booktabs}}
\usepackage{{pdflscape}}
\begin{{document}}
\thispagestyle{{empty}}
\begin{{landscape}}
{}
\end{{landscape}}
\end{{document}}
'''

path_breast_latex= os.path.join(path_prefix_results, 'breast_r2.tex')
path_thigh_latex= os.path.join(path_prefix_results, 'thigh_r2.tex')
path_breast_rmse_latex= os.path.join(path_prefix_results, 'breast_rmse.tex')
path_thigh_rmse_latex= os.path.join(path_prefix_results, 'thigh_rmse.tex')

In [3]:
def test_diff_cross(df, row='multi-atlas'):
    tmp= pd.DataFrame(index=df.columns, columns=df.columns)
    for i in df.columns:
        for j in df.columns:
            tmp.loc[i,j]= ttest_rel(np.abs(df.loc[row, i]), np.abs(df.loc[row, j]))[1]
    return tmp

def test_diff(df, col='multi-atlas'):
    tmp= pd.DataFrame(index=df.index, columns=df.columns)
    for i in df.index:
        for j in df.columns:
            tmp.loc[i,j]= ttest_rel(np.abs(df.loc[i, j]), np.abs(df.loc[col, j]))[1]
    tmp= tmp.drop(col, axis='rows')
    return tmp

def test_compare(df0, df1):
    tmp= pd.DataFrame(index=df0.index, columns=df0.columns)
    for i in df0.index:
        for j in df0.columns:
            tmp.loc[i,j]= ttest_rel(np.abs(df0.loc[i, j]), np.abs(df1.loc[i, j]))[1]
    return tmp

## Load data 

In [4]:
data_fs= pickle.load(open(os.path.join(path_prefix_results,'results.pickle'), 'rb'))
data_no_fs= pickle.load(open(os.path.join(path_prefix_results,'results_no_fs.pickle'), 'rb'))

In [5]:
thigh_fs= data_fs[data_fs['dataset'] == 'thigh']
breast_fs= data_fs[data_fs['dataset'] == 'breast']

thigh_no_fs= data_no_fs[data_no_fs['dataset'] == 'thigh']
breast_no_fs= data_no_fs[data_no_fs['dataset'] == 'breast']

In [6]:
print(sorted(thigh_fs['type'].unique()))

['074k', '075a', '075k', '077k', '079k', '080k', '082k', '083a', '083k', '084a', '087f', '088f', '088k', '090a', '091f', '092k', 'all', 'mean_mask']


In [7]:
rename_dict_regr= {'KNNR_Objective': 'kNN', 'LinearRegression_Objective': 'linear', 'PLSRegression_Objective': 'PLS', 'RidgeRegression_Objective': 'ridge', 'LassoRegression_Objective': 'lasso'}
#rename_dict_type= {'203a': 'atlas 1', '203k': 'atlas 2', '204f': 'atlas 3', '206k': 'atlas 4', '208f': 'atlas 5', 'all': 'multi-atlas', 'mean_mask': 'mean'}
#row_order= ['atlas 1', 'atlas 2', 'atlas 3', 'atlas 4', 'atlas 5', 'mean', 'multi-atlas']
rename_dict_type= sorted(thigh_fs['type'].unique())
rename_dict_type= dict(map(lambda x, i: (x, f"atlas {i+1}"), rename_dict_type,  range(len(rename_dict_type))))
rename_dict_type['all']= "multi-atlas"
rename_dict_type['mean_mask'] = "mean"
print(rename_dict_type)
row_order= list(rename_dict_type.values())
col_order= ['linear', 'PLS', 'lasso', 'ridge', 'kNN']

{'074k': 'atlas 1', '075a': 'atlas 2', '075k': 'atlas 3', '077k': 'atlas 4', '079k': 'atlas 5', '080k': 'atlas 6', '082k': 'atlas 7', '083a': 'atlas 8', '083k': 'atlas 9', '084a': 'atlas 10', '087f': 'atlas 11', '088f': 'atlas 12', '088k': 'atlas 13', '090a': 'atlas 14', '091f': 'atlas 15', '092k': 'atlas 16', 'all': 'multi-atlas', 'mean_mask': 'mean'}


## Thigh $r^2$ results

In [8]:
thigh_fs_results= thigh_fs.pivot(index='type', columns='model', values='r2_1')
thigh_fs_results= thigh_fs_results.rename(rename_dict_regr, axis='columns')
thigh_fs_results= thigh_fs_results.rename(rename_dict_type)
thigh_fs_results= thigh_fs_results.apply(lambda x: np.round(x, 4))
thigh_fs_results= thigh_fs_results.loc[row_order, col_order]

thigh_no_fs_results= thigh_no_fs.pivot(index='type', columns='model', values='r2_1')
thigh_no_fs_results= thigh_no_fs_results.rename(rename_dict_regr, axis='columns')
thigh_no_fs_results= thigh_no_fs_results.rename(rename_dict_type)
thigh_no_fs_results= thigh_no_fs_results.apply(lambda x: np.round(x, 4))
thigh_no_fs_results= thigh_no_fs_results.loc[row_order, col_order]

thigh_all= pd.concat([thigh_fs_results, thigh_no_fs_results], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
thigh_all.columns= columns

print(thigh_all.to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
 & \multicolumn{5}{r}{feature selection} & \multicolumn{5}{r}{no feature selection} \\
 & linear & PLS & lasso & ridge & kNN & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  &  &  &  &  &  \\
\midrule
atlas 1 & 0.577600 & 0.604700 & 0.579800 & 0.592800 & 0.621100 & 0.024000 & 0.493400 & 0.526500 & 0.461400 & 0.371900 \\
atlas 2 & 0.589300 & 0.588300 & 0.589900 & 0.596400 & 0.659400 & -1.587100 & 0.503400 & 0.456200 & 0.486400 & 0.403900 \\
atlas 3 & 0.590000 & 0.594700 & 0.589700 & 0.590100 & 0.596900 & 0.057300 & 0.507600 & 0.507800 & 0.489800 & 0.440300 \\
atlas 4 & 0.576000 & 0.583300 & 0.587500 & 0.576100 & 0.653200 & 0.028200 & 0.476900 & 0.485700 & 0.454100 & 0.382000 \\
atlas 5 & 0.517000 & 0.532000 & 0.584600 & 0.543700 & 0.599000 & -0.107800 & 0.434200 & 0.362700 & 0.472700 & 0.364900 \\
atlas 6 & 0.567900 & 0.564000 & 0.557900 & 0.631500 & 0.574000 & 0.211800 & 0.442300 & 0.478800 & 0.521100 & 0.393100 \\
atlas 7 & 0.542100 & 0

In [9]:
with open(path_thigh_latex, 'w') as f:
    f.write(latex_template.format(thigh_all.to_latex()))

## Thigh RMSE results

In [10]:
thigh_fs_results_rmse= thigh_fs.pivot(index='type', columns='model', values='rmse_1')
thigh_fs_results_rmse= thigh_fs_results_rmse.rename(rename_dict_regr, axis='columns')
thigh_fs_results_rmse= thigh_fs_results_rmse.rename(rename_dict_type)
thigh_fs_results_rmse= thigh_fs_results_rmse.apply(lambda x: np.round(x, 4))
thigh_fs_results_rmse= thigh_fs_results_rmse.loc[row_order, col_order]

thigh_no_fs_results_rmse= thigh_no_fs.pivot(index='type', columns='model', values='rmse_1')
thigh_no_fs_results_rmse= thigh_no_fs_results_rmse.rename(rename_dict_regr, axis='columns')
thigh_no_fs_results_rmse= thigh_no_fs_results_rmse.rename(rename_dict_type)
thigh_no_fs_results_rmse= thigh_no_fs_results_rmse.apply(lambda x: np.round(x, 4))
thigh_no_fs_results_rmse= thigh_no_fs_results_rmse.loc[row_order, col_order]

thigh_all_rmse= pd.concat([thigh_fs_results_rmse, thigh_no_fs_results_rmse], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
thigh_all_rmse.columns= columns

print(thigh_all_rmse.to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
 & \multicolumn{5}{r}{feature selection} & \multicolumn{5}{r}{no feature selection} \\
 & linear & PLS & lasso & ridge & kNN & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  &  &  &  &  &  \\
\midrule
atlas 1 & 12.227600 & 11.828300 & 12.194800 & 12.005200 & 11.580900 & 18.586800 & 13.390100 & 12.945700 & 13.807300 & 14.909700 \\
atlas 2 & 12.056500 & 12.071000 & 12.047600 & 11.952200 & 10.979900 & 30.260800 & 13.257500 & 13.874200 & 13.482500 & 14.525400 \\
atlas 3 & 12.045900 & 11.977100 & 12.050800 & 12.045600 & 11.944700 & 18.266500 & 13.201200 & 13.199100 & 13.438000 & 14.074400 \\
atlas 4 & 12.249800 & 12.144900 & 12.082500 & 12.249700 & 11.079200 & 18.546600 & 13.606500 & 13.492700 & 13.899900 & 14.790300 \\
atlas 5 & 13.074600 & 12.870300 & 12.125700 & 12.708000 & 11.914300 & 19.801900 & 14.151800 & 15.019400 & 13.661300 & 14.993100 \\
atlas 6 & 12.366900 & 12.423200 & 12.509100 & 11.419900 & 12.280000 & 16.703100 & 14.050300 & 1

In [11]:
with open(path_thigh_rmse_latex, 'w') as f:
    f.write(latex_template.format(thigh_all_rmse.to_latex()))

## t-test thigh

In [12]:
thigh_fs['diff']= (thigh_fs['y_pred_1'] - thigh_fs['y_test_1'])**2
thigh_fs_test= thigh_fs.pivot(index='type', columns='model', values='diff')
thigh_fs_test= thigh_fs_test.rename(rename_dict_regr, axis='columns')
thigh_fs_test= thigh_fs_test.rename(rename_dict_type)
thigh_fs_test= thigh_fs_test.loc[row_order, col_order]

thigh_no_fs['diff']= (thigh_no_fs['y_pred_1'] - thigh_no_fs['y_test_1'])**2
thigh_no_fs_test= thigh_no_fs.pivot(index='type', columns='model', values='diff')
thigh_no_fs_test= thigh_no_fs_test.rename(rename_dict_regr, axis='columns')
thigh_no_fs_test= thigh_no_fs_test.rename(rename_dict_type)
thigh_no_fs_test= thigh_no_fs_test.loc[row_order, col_order]

In [13]:
print(test_compare(thigh_fs_test, thigh_no_fs_test).applymap(lambda x: '%.0e' % x).to_latex())

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  \\
\midrule
atlas 1 & 3e-44 & 3e-28 & 3e-13 & 2e-33 & 7e-32 \\
atlas 2 & 7e-09 & 3e-12 & 2e-20 & 3e-16 & 9e-34 \\
atlas 3 & 5e-53 & 1e-16 & 1e-14 & 1e-21 & 4e-20 \\
atlas 4 & 3e-47 & 4e-23 & 5e-34 & 1e-15 & 1e-26 \\
atlas 5 & 1e-09 & 2e-13 & 3e-18 & 9e-07 & 2e-26 \\
atlas 6 & 3e-16 & 2e-10 & 9e-05 & 1e-13 & 9e-22 \\
atlas 7 & 8e-37 & 7e-25 & 4e-13 & 7e-13 & 4e-21 \\
atlas 8 & 1e-37 & 7e-12 & 7e-14 & 1e-19 & 4e-15 \\
atlas 9 & 1e-16 & 5e-08 & 2e-17 & 2e-26 & 1e-42 \\
atlas 10 & 3e-41 & 5e-27 & 4e-25 & 1e-12 & 1e-29 \\
atlas 11 & 3e-14 & 5e-21 & 1e-10 & 1e-17 & 2e-43 \\
atlas 12 & 2e-34 & 6e-11 & 5e-18 & 1e-12 & 4e-35 \\
atlas 13 & 4e-39 & 2e-21 & 7e-11 & 2e-22 & 2e-33 \\
atlas 14 & 3e-60 & 2e-15 & 6e-19 & 5e-15 & 6e-32 \\
atlas 15 & 2e-49 & 2e-18 & 4e-22 & 3e-31 & 2e-29 \\
atlas 16 & 2e-50 & 2e-17 & 2e-18 & 4e-32 & 1e-39 \\
multi-atlas & 2e-58 & 2e-68 & 3e-95 & 1e-78 & 3e-50 \\
mean & 7e-2

In [14]:
print(test_diff(thigh_fs_test).applymap(lambda x: '%.0e' % x).to_latex())

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  \\
\midrule
atlas 1 & 2e-35 & 1e-72 & 5e-119 & 3e-111 & 6e-02 \\
atlas 2 & 7e-32 & 3e-81 & 6e-131 & 9e-96 & 7e-01 \\
atlas 3 & 9e-37 & 5e-78 & 1e-118 & 3e-111 & 4e-06 \\
atlas 4 & 3e-41 & 1e-76 & 1e-131 & 1e-103 & 9e-01 \\
atlas 5 & 1e-58 & 2e-111 & 2e-100 & 3e-137 & 1e-04 \\
atlas 6 & 3e-44 & 3e-104 & 5e-158 & 3e-104 & 2e-11 \\
atlas 7 & 1e-49 & 1e-83 & 1e-139 & 4e-108 & 7e-03 \\
atlas 8 & 2e-56 & 7e-91 & 3e-99 & 2e-91 & 2e-36 \\
atlas 9 & 8e-37 & 2e-83 & 3e-79 & 4e-88 & 2e-03 \\
atlas 10 & 8e-36 & 1e-54 & 5e-89 & 5e-77 & 1e-03 \\
atlas 11 & 9e-46 & 1e-74 & 1e-130 & 3e-103 & 5e-23 \\
atlas 12 & 9e-46 & 2e-87 & 3e-125 & 2e-126 & 3e-07 \\
atlas 13 & 2e-53 & 1e-86 & 2e-151 & 2e-117 & 6e-12 \\
atlas 14 & 4e-16 & 2e-61 & 6e-106 & 2e-98 & 1e-06 \\
atlas 15 & 2e-22 & 1e-49 & 4e-120 & 1e-113 & 4e-02 \\
atlas 16 & 5e-35 & 3e-72 & 3e-109 & 3e-110 & 2e-05 \\
mean & 2e-41 & 3e-69 & 2e-130 & 1e-116 &

In [15]:
print(test_diff_cross(thigh_fs_test.loc[['multi-atlas']]).applymap(lambda x: '%.0e' % x).to_latex())

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
model &  &  &  &  &  \\
\midrule
linear & nan & 9e-35 & 3e-73 & 5e-64 & 4e-11 \\
PLS & 9e-35 & nan & 2e-65 & 1e-47 & 3e-67 \\
lasso & 3e-73 & 2e-65 & nan & 5e-08 & 4e-103 \\
ridge & 5e-64 & 1e-47 & 5e-08 & nan & 5e-99 \\
kNN & 4e-11 & 3e-67 & 4e-103 & 5e-99 & nan \\
\bottomrule
\end{tabular}



## Breast $r^2$ results

In [16]:
breast_fs_results= breast_fs.pivot(index='type', columns='model', values='r2_1')
breast_fs_results= breast_fs_results.rename(rename_dict_regr, axis='columns')
breast_fs_results= breast_fs_results.rename(rename_dict_type)
breast_fs_results= breast_fs_results.apply(lambda x: np.round(x, 4))
breast_fs_results= breast_fs_results.loc[row_order, col_order]

breast_no_fs_results= breast_no_fs.pivot(index='type', columns='model', values='r2_1')
breast_no_fs_results= breast_no_fs_results.rename(rename_dict_regr, axis='columns')
breast_no_fs_results= breast_no_fs_results.rename(rename_dict_type)
breast_no_fs_results= breast_no_fs_results.apply(lambda x: np.round(x, 4))
breast_no_fs_results= breast_no_fs_results.loc[row_order, col_order]

breast_all= pd.concat([breast_fs_results, breast_no_fs_results], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
breast_all.columns= columns

print(breast_all.to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
 & \multicolumn{5}{r}{feature selection} & \multicolumn{5}{r}{no feature selection} \\
 & linear & PLS & lasso & ridge & kNN & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  &  &  &  &  &  \\
\midrule
atlas 1 & 0.885900 & 0.881600 & 0.879000 & 0.894300 & 0.849600 & 0.752000 & 0.862600 & 0.859900 & 0.871800 & 0.652100 \\
atlas 2 & 0.877900 & 0.882100 & 0.871100 & 0.849900 & 0.831100 & 0.721100 & 0.831400 & 0.821400 & 0.827200 & 0.650000 \\
atlas 3 & 0.918400 & 0.907000 & 0.918600 & 0.913600 & 0.814100 & 0.615000 & 0.877200 & 0.873500 & 0.877800 & 0.671400 \\
atlas 4 & 0.883000 & 0.901700 & 0.880200 & 0.911100 & 0.760900 & 0.817700 & 0.860800 & 0.878600 & 0.880500 & 0.635800 \\
atlas 5 & 0.910000 & 0.901700 & 0.896100 & 0.905900 & 0.826700 & 0.646100 & 0.865700 & 0.858500 & 0.863300 & 0.689700 \\
atlas 6 & 0.894200 & 0.895800 & 0.893600 & 0.895800 & 0.835000 & 0.779000 & 0.878700 & 0.878500 & 0.879200 & 0.712300 \\
atlas 7 & 0.910400 & 0.9

In [17]:
with open(path_breast_latex, 'w') as f:
    f.write(latex_template.format(breast_all.to_latex()))

## Breast RMSE results

In [18]:
breast_fs_results_rmse= breast_fs.pivot(index='type', columns='model', values='rmse_1')
breast_fs_results_rmse= breast_fs_results_rmse.rename(rename_dict_regr, axis='columns')
breast_fs_results_rmse= breast_fs_results_rmse.rename(rename_dict_type)
breast_fs_results_rmse= breast_fs_results_rmse.apply(lambda x: np.round(x, 4))
breast_fs_results_rmse= breast_fs_results_rmse.loc[row_order, col_order]

breast_no_fs_results_rmse= breast_no_fs.pivot(index='type', columns='model', values='rmse_1')
breast_no_fs_results_rmse= breast_no_fs_results_rmse.rename(rename_dict_regr, axis='columns')
breast_no_fs_results_rmse= breast_no_fs_results_rmse.rename(rename_dict_type)
breast_no_fs_results_rmse= breast_no_fs_results_rmse.apply(lambda x: np.round(x, 4))
breast_no_fs_results_rmse= breast_no_fs_results_rmse.loc[row_order, col_order]

breast_all_rmse= pd.concat([breast_fs_results_rmse, breast_no_fs_results_rmse], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
breast_all_rmse.columns= columns

print(breast_all_rmse.to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
 & \multicolumn{5}{r}{feature selection} & \multicolumn{5}{r}{no feature selection} \\
 & linear & PLS & lasso & ridge & kNN & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  &  &  &  &  &  \\
\midrule
atlas 1 & 19.542400 & 19.907000 & 20.121800 & 18.806800 & 22.431400 & 28.803300 & 21.444500 & 21.654000 & 20.710200 & 34.118400 \\
atlas 2 & 20.212500 & 19.865800 & 20.763700 & 22.407300 & 23.770300 & 30.546800 & 23.747900 & 24.443500 & 24.045900 & 34.223400 \\
atlas 3 & 16.519500 & 17.635600 & 16.500900 & 17.002800 & 24.940100 & 35.892600 & 20.267700 & 20.576000 & 20.221200 & 33.159700 \\
atlas 4 & 19.786000 & 18.139400 & 20.021000 & 17.247800 & 28.281600 & 24.698900 & 21.582300 & 20.153800 & 19.998200 & 34.908300 \\
atlas 5 & 17.352500 & 18.136100 & 18.647300 & 17.746900 & 24.076800 & 34.411800 & 21.199000 & 21.761900 & 21.383100 & 32.223800 \\
atlas 6 & 18.818500 & 18.672200 & 18.869500 & 18.669700 & 23.493900 & 27.195000 & 20.142700 & 2

In [19]:
with open(path_breast_rmse_latex, 'w') as f:
    f.write(latex_template.format(breast_all_rmse.to_latex()))

## t-test breast

In [20]:
breast_fs['diff']= (breast_fs['y_pred_1'] - breast_fs['y_test_1'])**2
breast_fs_test= breast_fs.pivot(index='type', columns='model', values='diff')
breast_fs_test= breast_fs_test.rename(rename_dict_regr, axis='columns')
breast_fs_test= breast_fs_test.rename(rename_dict_type)
breast_fs_test= breast_fs_test.loc[row_order, col_order]

breast_no_fs['diff']= (breast_no_fs['y_pred_1'] - breast_no_fs['y_test_1'])**2
breast_no_fs_test= breast_no_fs.pivot(index='type', columns='model', values='diff')
breast_no_fs_test= breast_no_fs_test.rename(rename_dict_regr, axis='columns')
breast_no_fs_test= breast_no_fs_test.rename(rename_dict_type)
breast_no_fs_test= breast_no_fs_test.loc[row_order, col_order]

In [21]:
print(test_compare(breast_fs_test, breast_no_fs_test).applymap(lambda x: '%.0e' % x).to_latex())

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  \\
\midrule
atlas 1 & 7e-36 & 5e-08 & 4e-08 & 3e-16 & 5e-36 \\
atlas 2 & 3e-46 & 8e-13 & 1e-10 & 3e-03 & 2e-40 \\
atlas 3 & 2e-09 & 2e-10 & 8e-26 & 1e-16 & 2e-16 \\
atlas 4 & 7e-10 & 6e-22 & 7e-01 & 7e-19 & 5e-30 \\
atlas 5 & 5e-12 & 2e-13 & 2e-10 & 2e-13 & 2e-42 \\
atlas 6 & 2e-38 & 2e-16 & 2e-09 & 2e-23 & 3e-30 \\
atlas 7 & 2e-43 & 7e-18 & 3e-24 & 8e-19 & 3e-62 \\
atlas 8 & 2e-33 & 1e-06 & 2e-18 & 2e-24 & 1e-63 \\
atlas 9 & 9e-25 & 4e-11 & 6e-16 & 5e-22 & 5e-51 \\
atlas 10 & 1e-18 & 2e-16 & 2e-30 & 3e-20 & 2e-32 \\
atlas 11 & 1e-23 & 3e-10 & 2e-11 & 4e-04 & 1e-32 \\
atlas 12 & 4e-16 & 5e-15 & 5e-08 & 2e-13 & 2e-26 \\
atlas 13 & 5e-40 & 2e-08 & 3e-11 & 2e-07 & 7e-52 \\
atlas 14 & 4e-36 & 1e-14 & 8e-24 & 2e-32 & 6e-41 \\
atlas 15 & 7e-12 & 8e-16 & 3e-29 & 3e-29 & 3e-55 \\
atlas 16 & 6e-11 & 8e-23 & 6e-32 & 6e-28 & 2e-56 \\
multi-atlas & 6e-69 & 2e-59 & 8e-69 & 1e-82 & 2e-51 \\
mean & 3e-2

In [22]:
print(test_diff(breast_fs_test).applymap(lambda x: '%.0e' % x).to_latex())

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
type &  &  &  &  &  \\
\midrule
atlas 1 & 1e-57 & 1e-58 & 2e-115 & 1e-108 & 7e-02 \\
atlas 2 & 1e-55 & 4e-56 & 2e-99 & 9e-88 & 3e-08 \\
atlas 3 & 4e-32 & 5e-62 & 4e-112 & 6e-96 & 6e-12 \\
atlas 4 & 9e-66 & 1e-58 & 3e-78 & 1e-128 & 2e-35 \\
atlas 5 & 3e-43 & 3e-55 & 1e-91 & 1e-135 & 6e-16 \\
atlas 6 & 1e-56 & 3e-56 & 9e-118 & 3e-112 & 2e-05 \\
atlas 7 & 4e-40 & 5e-59 & 1e-92 & 1e-101 & 1e-01 \\
atlas 8 & 6e-97 & 1e-76 & 7e-124 & 3e-99 & 5e-30 \\
atlas 9 & 2e-59 & 1e-58 & 4e-98 & 2e-116 & 4e-10 \\
atlas 10 & 1e-31 & 2e-42 & 6e-81 & 3e-84 & 2e-24 \\
atlas 11 & 2e-45 & 1e-54 & 6e-116 & 2e-103 & 2e-01 \\
atlas 12 & 2e-48 & 2e-46 & 4e-76 & 6e-101 & 3e-14 \\
atlas 13 & 1e-50 & 1e-52 & 5e-80 & 1e-84 & 4e-04 \\
atlas 14 & 6e-38 & 8e-63 & 7e-98 & 3e-94 & 2e-14 \\
atlas 15 & 9e-34 & 1e-53 & 3e-84 & 2e-98 & 2e-16 \\
atlas 16 & 3e-48 & 5e-55 & 1e-101 & 9e-103 & 9e-03 \\
mean & 1e-41 & 7e-51 & 3e-94 & 8e-108 & 8e-01 \\
\b

In [23]:
print(test_diff_cross(breast_fs_test.loc[['multi-atlas']]).applymap(lambda x: '%.0e' % x).to_latex())

\begin{tabular}{llllll}
\toprule
model & linear & PLS & lasso & ridge & kNN \\
model &  &  &  &  &  \\
\midrule
linear & nan & 6e-01 & 5e-40 & 2e-44 & 3e-77 \\
PLS & 6e-01 & nan & 1e-67 & 2e-77 & 2e-83 \\
lasso & 5e-40 & 1e-67 & nan & 7e-12 & 2e-117 \\
ridge & 2e-44 & 2e-77 & 7e-12 & nan & 3e-119 \\
kNN & 3e-77 & 2e-83 & 2e-117 & 3e-119 & nan \\
\bottomrule
\end{tabular}

